In [ ]:
from dotenv import load_dotenv
import os

# Cargar las variables del archivo .env
load_dotenv()

# Obtener la clave de OpenAI
openai_api_key = os.getenv("OPENAI_API_KEY")

# Verifica que la clave se ha cargado correctamente
if openai_api_key:
    print("La clave de OpenAI se ha cargado correctamente.")
else:
    print("Error al cargar la clave de OpenAI.")

In [2]:
from openai import OpenAI
client=OpenAI(api_key=openai_api_key)
MODEL='gpt-4-turbo'

In [ ]:
import yt_dlp
import re

# URL del vídeo de YouTube
url = "https://youtu.be/jWUerU_7kX4"

# Usar yt-dlp para obtener la información del vídeo
ydl_opts = {}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info_dict = ydl.extract_info(url, download=False)
    video_title = info_dict.get('title', None)
    video_views = info_dict.get('view_count', None)
    video_length = info_dict.get('duration', None)
    video_thumbnail = info_dict.get('thumbnail', None)

# Imprimir la información obtenida
print(f"Title: {video_title}")
print(f"Views: {video_views}")
print(f"Length: {video_length} seconds")
print(f"Thumbnail: {video_thumbnail}")

In [50]:
import re
# Función para limpiar el título del vídeo
def sanitize_filename(filename):
    # Reemplaza los espacios por guiones bajos
    filename = filename.replace(" ", "_")
    # Elimina caracteres no alfanuméricos, dejando solo letras, números y guiones bajos
    filename = re.sub(r'[^\w-]', '', filename)
    return filename


def descargar_solo_audio(url,output_dir="output"):
    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)
    # Función para limpiar el título del vídeo
    def sanitize_filename(filename):
        # Reemplaza los espacios por guiones bajos
        filename = filename.replace(" ", "_")
        # Elimina caracteres no alfanuméricos, dejando solo letras, números y guiones bajos
        filename = re.sub(r'[^\w-]', '', filename)
        return filename

    print("descargar_solo_audio...")

    # Obtener la información del vídeo y el título
    ydl_opts_info = {
        'quiet': True,  # Esta opción desactiva los mensajes de yt-dlp
        'no_warnings': True  # Desactiva las advertencias
    }
    with yt_dlp.YoutubeDL(ydl_opts_info) as ydl:
        info_dict = ydl.extract_info(url, download=False)
        video_title = info_dict.get('title', None)
    video_title = sanitize_filename(video_title)
    print(f"Title: {video_title}")

    # Opciones para descargar solo el audio
    ydl_opts = {
        'format': 'bestaudio/best',  # Descargar la mejor calidad de audio disponible
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',  # Puedes cambiar a 'wav' o 'm4a' si prefieres otro formato
            'preferredquality': '192',  # Calidad de audio
        }],
        'outtmpl': os.path.join(output_dir, f'{video_title}.%(ext)s'),  # Guardar el archivo con el título limpio
        'quiet': True,  # Esta opción desactiva los mensajes de yt-dlp
        'no_warnings': True  # Desactiva las advertencias
    }

    # Descargar el audio
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    
    filename = f"{output_dir}/{video_title}.mp3"
    print(f"Descarga de audio completa. Archivo guardado como: {filename}")
    return filename

def transcribe(audio_file,output_dir="transcribed_output"):
    if not os.path.isfile(audio_file):
        print(f"El archivo {audio_file} no existe.")
        return  False
    
    # Crear el directorio de salida si no existe
    os.makedirs(output_dir, exist_ok=True)
    
    with open(audio_file, 'rb') as f:
        print("Transcribiendo audio...",end='')
        transcript = client.audio.transcriptions.create(
            model='whisper-1',
            file=f
        )
        print(f"Transcripción completa {transcript.text}")
        name,extension = os.path.splitext(audio_file)
        transcribed_file = f"transcribed_{name}.txt"
        print(f"Archivo de transcripción guardado como: {transcribed_file}")
        with open(transcribed_file, 'w') as transcription_file:
            transcription_file.write(transcript.text)
    return transcribed_file

def resumen_transcripcion(file,idioma="Castellano",formato="markdown"):
    #enviar la transcripción a openAI para obtener el resumen
    MODEL='gpt-4o'
    system_role=f'''
    Eres un asistente para resumir. Te pasan la transcripción de un vídeo y respondes
    con un texto donde explicas los contenidos del video.
    Empiezas con una introducción, siguies con los apartados principales y terminas con una conclusión.
    Devuelves el texto en formato markdown y solamente escribes el contenido del texto.
    Siempre respondes en {idioma}
    '''

    # Instrucciones para el asistente mejores
    system_role = f'''
    Eres un asistente experto en resumir vídeos. Te pasarán la transcripción de un vídeo,
    y debes devolver un resumen claro y estructurado. Sigue estas instrucciones:
    
    1. Introducción breve que resuma el tema general del vídeo.
    2. Divide el resumen en secciones basadas en los temas principales. Pon ejemplos de los apartados que se encuentran en el video.
    3. Termina con una conclusión o cierre.
    
    El resumen debe estar en {idioma} y escrito en {formato}.
    '''

    # Leer el contenido de cada archivo y agregarlo a la transcripción total
    texto_a_resumir=""
    with open(file, 'r') as f:
        content = f.read()
        texto_a_resumir += content + "\n"
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {'role':'system','content':system_role},
            {'role':'user','content':texto_a_resumir},
        ],
    )
    directory=file.split('/')[0]
    base_name=file.split('/')[1]
    file_resumed=f"{directory}/resumed_{base_name}"
    print(f"Archivo de resumen guardado como: {file_resumed}")
    with open(file_resumed, 'w') as transcription_file:
        transcription_file.write(response.choices[0].message.content)
    return response.choices[0].message.content

def guion_video(file,idioma="Castellano"):
    #enviar la transcripción a openAI para obtener el resumen
    MODEL='gpt-4o'
    system_role=f'''
    Eres un guionista para vídeos de youtube. Te pasan la transcripción de un vídeo y respondes
    con un guión del vídeo en el idioma {idioma} con frases elaboradas.
    Tienes que presentar un guión con los minutos más exactos para que 
    se pueda grabar un nuevo vídeo con el mismo contenido pero en el idioma {idioma}.
    Devuelves el texto en formato markdown.
    
    '''

    system_role = f'''
    Eres un guionista profesional para vídeos de YouTube. 
    Te pasan la transcripción de un vídeo y debes generar un guion con frases elaboradas, listo para regrabar el vídeo.
    
    El guion debe estar en el idioma {idioma}, debe incluir marcadores de tiempo aproximados en minutos y segundos, 
    y estar bien estructurado para su fácil lectura. 
    
    Devuelve el guion en formato markdown y asegúrate de mejorar las frases para que el vídeo final sea más atractivo.
    '''

    # Leer el contenido de cada archivo y agregarlo a la transcripción total
    texto_a_resumir=""
    with open(file, 'r') as f:
        content = f.read()
        texto_a_resumir += content + "\n"
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {'role':'system','content':system_role},
            {'role':'user','content':texto_a_resumir},
        ],
    )
    directory=file.split('/')[0]
    base_name=file.split('/')[1]
    file_resumed=f"{directory}/guion_{base_name}"
    print(f"Archivo de guion guardado como: {file_resumed}")
    with open(file_resumed, 'w') as transcription_file:
        transcription_file.write(response.choices[0].message.content)
    return response.choices[0].message.content


In [ ]:
audio_filename=descargar_solo_audio(url)
print(f"Audio filename: {audio_filename}")

In [ ]:
audio_filename


In [ ]:
transcribed_filename=transcribe(audio_filename)

In [ ]:
resumed_video_filename=resumen_transcripcion(transcribed_filename)

In [ ]:
guion_video_filename=guion_video(transcribed_filename)